In [6]:
import pandas as pd
import numpy as np
import matplotlib as mlt

In [7]:
# Import DataFrame
rawData = pd.read_csv("apple_music.csv")
print("shape of the original data:", rawData.shape, "\n")

# Make a Working Copy & select olumns that will be used
wrkData = rawData.copy()
wrkData = wrkData[['Title', 'Artist', 'Composer', 'Album', 'Genre', 
                   'Track Year', 'Track Number On Album', 'Track Count On Album', 
                   'Track Duration', 'Track Play Count', 'Date Added To Library', 
                   'Last Played Date', 'Skip Count']]
print(wrkData.loc[0:1, ["Title", "Artist", "Last Played Date"]])

shape of the original data: (3119, 53) 

                     Title       Artist      Last Played Date
0  Never Gonna Give You Up  Rick Astley  2021-12-10T05:47:34Z
1              Lay Me Down    Sam Smith                   NaN


In [8]:
# Raname columns (inplace = True modifies the instance itself)
wrkData.rename(
    columns={"Title" : "title", "Artist" : "artist", "Composer" : "composer",
             "Album" : "album", "Genre" : "genre", "Track Year": "track year", 
             "Track Number On Album" : "track no.", "Track Count On Album" : "track count", 
             "Track Duration" : "duration", "Track Play Count" : "play count", 
             "Date Added To Library" : "added date", "Last Played Date" : "last played", 
             "Skip Count" : "skipped"}, inplace = True)
print(wrkData.columns)

Index(['title', 'artist', 'composer', 'album', 'genre', 'track year',
       'track no.', 'track count', 'duration', 'play count', 'added date',
       'last played', 'skipped'],
      dtype='object')


In [9]:
# remove NAs in artist column
